In [1]:
import pickle
import networkx as nx
import json
import pandas as pd
from pyvis.network import Network

f= open('data/grafo_autores.json') 
data_autores = json.load(f)
f.close()

f= open('data/lista_papers.json')
data_papers = json.load(f)
f.close()

f= open('data/grafo_instituciones.json') 
data_inst = json.load(f)
f.close()

f= open('data/lista_papers_instituciones.json') 
data_paper_inst = json.load(f)
f.close()

with open("data/cluster_authorrank_2000.pkl", "rb") as f:
        data_cluster = pickle.load(f)

with open("data/cluster_filiacion_authorrank_500.pkl", "rb") as f:
        data_cluster_filiacion = pickle.load(f)

salida_articulos = pd.read_excel("data/output_articles.xlsx")
salidas_topicos = pd.read_excel("data/salidas_topicos_CB.xlsx")
salida_autores = pd.read_excel("data/salida_autores.xlsx")


#El nuevo data_papers viene con los values de nodos en entero, hay que convertirlos a str para que el codigo funcione sin modificaciones.
data_papers_str = {}
for key, value in data_papers.items():
    data_papers_str[key] = [str(node) for node in value]

data_papers = data_papers_str

In [2]:
# La enumeracion del cluster va en orden descendente de acuerdo al 
# orden de los grandes productores

Enumerar_cluster = {}

for i, ele in enumerate(data_cluster, start = 1):
    Enumerar_cluster[i] = ele.split('_')

Enumerar_nodo = {}
for key, lista_nodos in Enumerar_cluster.items():
    for nodo in lista_nodos:
        Enumerar_nodo[nodo] = key

In [3]:
# Encontrar las conexciones entre comunidades

n_comunidades = 200



# Necesito una lista de total de artículos por comunidad
list_paper_comunidad = {}

for i in range(1,n_comunidades):
    total_papers = []
    for node in Enumerar_cluster[i]:
        paper_node = data_autores[node]['articulos']
        total_papers = total_papers + paper_node

    list_paper_comunidad[i] = set(total_papers)


# Necesito contar los paper en comun
import itertools

inicio = []
final = []
articulos = []

combinaciones = list(itertools.combinations(list_paper_comunidad.keys(), 2))

for x,y in combinaciones:
    paper_comn1 = list(list_paper_comunidad[x])
    paper_comn2 = list(list_paper_comunidad[y])

    paper_comun = [elemento for elemento in paper_comn1 if elemento in paper_comn2]
    # if len(paper_comun) != 0:
    inicio.append(str(x))
    final.append(str(y))
    articulos.append(len(paper_comun))

    # print(f'papers en comun entre comunidad {x}, {y}, {len(paper_comun)}')

# Debemos contar los extranjeros por comunidad, ie, los no Chilenos

contar_extranjeros = {}
for id, nodes in Enumerar_cluster.items():
    count = 0
    for node in nodes:
        if data_autores[node]['country'] == 'chile':
            # print(f'{id}, {data_autores[node]["nombre_autor"]}')
            count += 1
    contar_extranjeros[id] = {"extranjeros": len(nodes) - count,
                              "chile": count,
                              "mayoria": 'si' if ((len(nodes) - count)/len(nodes))*100 > 50 else 'no'}
    
# mayoria se define como un 50% o más de extranjeros
# contar_extranjeros


In [4]:
comn_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True, cdn_resources='in_line')
# comn_net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white")
# edge_data = zip(inicio, final, articulos)

# Agregar nodos y bordes a la red
edge_data = zip(inicio, final, articulos)

for src, dst, w in edge_data:
    comn_net.add_node(src, src, title=src)
    comn_net .add_node(dst, dst, title=dst)
    if w != 0:
        comn_net.add_edge(src, dst, value=w)  

In [5]:
for node in comn_net.nodes:
    
    int_node = int(node["id"])
    n = len(list_paper_comunidad[int_node])
    
    node["label"] = node["id"]
    node["value"] = 20 * n
    node["font"] = {"size":  n, "color": "white"}

    cluster = Enumerar_cluster[int_node]
    lista_nombres = "\n"
    for node_c in cluster:
        n_art = data_autores[node_c]['n_articulos']
        lista_nombres += data_autores[node_c]['nombre_autor'] + ": número artículos " + str(n_art) + " \n"

    node["title"] = "Comunidad " + str(int_node) +  "\n Total artículos " + str(n) + " \n Miembros de la comunidad: " + lista_nombres 
 

    if contar_extranjeros[int_node]["mayoria"] == 'si':
        node["color"] = "Green"
    else:
        node["color"] = 'Red'

In [6]:
comn_net.show('GrafoComunidadAutores.html')

GrafoComunidadAutores.html
